In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## The Beginner's Guide to Key Functionality

| | |
|-|-|
| Author(s) | [Laxmi Harikumar](https://github.com/laxmi-genai) |

## Overview

[Gemini 2.0 Flash-Lite](https://cloud.google.com/vertex-ai/generative-ai/docs/gemini-v2#2.0-flash-lite) is a new multimodal generative ai model from the Gemini family developed by [Google DeepMind](https://deepmind.google/). It is Google's fastest and most cost efficient Flash model. It's an upgrade path for 1.5 Flash users who want better quality for the same price and speed.

Gemini 2.0 Flash-Lite includes:

- Multimodal input, text output
- 1M token input context window
- 8k token output context window

### Pricing
Information on the pricing for Gemini 2.0 Flash is available on our [Pricing page](https://cloud.google.com/vertex-ai/generative-ai/pricing).


## Objective

In this notebook, you'll explore the examples for:

- Controlled Generation
- Automated Function Calling

## Getting Started

### Install Google Gen AI SDK for Python

In [ ]:
!pip install --upgrade --quiet google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 4.6 MB/s eta 0:00:00


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)
If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()


### Import libraries

In [ ]:
from IPython.display import Markdown, display
from google import genai
from google.genai.types import (
    GenerateContentConfig,
    Part,
    SafetySetting,
    Tool,
    Content
)
import os

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = ""  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

In [ ]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## Use the Gemini 2.0 Flash-Lite model

In [ ]:
MODEL_ID = "gemini-2.0-flash-lite-preview-02-05"  # @param {type: "string"}

## Safety Filters

The Gemini API provides safety filters that you can adjust across multiple filter categories to restrict or allow certain types of content. You can use these filters to adjust what's appropriate for your use case. See the Configure safety filters page for details.

When you make a request to Gemini, the content is analyzed and assigned a safety rating. You can inspect the safety ratings of the generated content by printing out the model responses.

You can use safety_settings to adjust the safety settings for each request you make to the API. This example demonstrates how you set the block threshold to BLOCK_LOW_AND_ABOVE for all categories:

In [ ]:
safety_settings = [
    SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
    SafetySetting(
        category="HARM_CATEGORY_HARASSMENT",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
    SafetySetting(
        category="HARM_CATEGORY_HATE_SPEECH",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
    SafetySetting(
        category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
]

## Configure model parameters

You can include parameter values in each call that you send to a model to control how the model generates a response. The model can generate different results for different parameter values. You can experiment with different model parameters to see how the results change.

- Learn more about [experimenting with parameter values](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/adjust-parameter-values).

- See a list of all [Gemini API parameters](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#parameters).

In [ ]:
generate_content_config=GenerateContentConfig(
        temperature=0.4,
        top_p=0.95,
        top_k=20,
        safety_settings=safety_settings
    )

## Controlled Generation

[Controlled generation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output) allows you to define a response schema to specify the structure of a model's output, the field names, and the expected data type for each field.

The response schema is specified in the `response_schema` parameter in `config`, and the model output will strictly follow that schema.

You can provide the schemas as [Pydantic](https://docs.pydantic.dev/) models or a [JSON](https://www.json.org/json-en.html) string and the model will respond as JSON or an [Enum](https://docs.python.org/3/library/enum.html) depending on the value set in `response_mime_type`.

In [ ]:
from pydantic import BaseModel


class Recipe(BaseModel):
    name: str
    description: str
    ingredients: list[str]


response = client.models.generate_content(
    model=MODEL_ID,
    contents="List a popular cookie recipe and its ingredients.",
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=Recipe,
    ),
)

print(response.text)

{
  "description": "This is a classic chocolate chip cookie recipe.",
  "ingredients": [
    "1 cup (2 sticks) unsalted butter, softened",
    "3/4 cup granulated sugar",
    "3/4 cup packed brown sugar",
    "1 teaspoon vanilla extract",
    "2 large eggs",
    "2 1/4 cups all-purpose flour",
    "1 teaspoon baking soda",
    "1 teaspoon salt",
    "2 cups chocolate chips"
  ],
  "name": "Chocolate Chip Cookies"
}


You can either parse the response string as JSON, or use the `parsed` field to get the response as an object or dictionary.

In [ ]:
parsed_response: Recipe = response.parsed
print(parsed_response)

name='Chocolate Chip Cookies' description='This is a classic chocolate chip cookie recipe.' ingredients=['1 cup (2 sticks) unsalted butter, softened', '3/4 cup granulated sugar', '3/4 cup packed brown sugar', '1 teaspoon vanilla extract', '2 large eggs', '2 1/4 cups all-purpose flour', '1 teaspoon baking soda', '1 teaspoon salt', '2 cups chocolate chips']


Controlled generation supports the following fields from the [Vertex AI schema](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output). If you use an unsupported field, Vertex AI can still handle your request but ignores the field.

- `anyOf`
- `enum`
- `format`
- `items`
- `maximum`
- `maxItems`
- 'minimum`
- 'minItems`
- 'nullable`
- 'properties`
- 'required`

In this example, you instruct the model to analyze product review data, extract key entities, perform sentiment classification (multiple choices), provide additional explanation, and output the results in JSON format.

In [ ]:
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "rating": {"type": "INTEGER"},
                "flavor": {"type": "STRING"},
                "sentiment": {
                    "type": "STRING",
                    "enum": ["POSITIVE", "NEGATIVE", "NEUTRAL"],
                },
                "explanation": {"type": "STRING"},
            },
            "required": ["rating", "flavor", "sentiment", "explanation"],
        },
    },
}

prompt = """
  Analyze the following product reviews, output the sentiment classification, and give an explanation.

  - "Absolutely loved it! Best ice cream I've ever had." Rating: 4, Flavor: Strawberry Cheesecake
  - "Quite good, but a bit too sweet for my taste." Rating: 1, Flavor: Mango Tango
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)

response_dict = response.parsed
display(response_dict)

[[{'explanation': 'The review expresses strong positive feelings and high satisfaction.',
   'flavor': 'Strawberry Cheesecake',
   'rating': 4,
   'sentiment': 'POSITIVE'},
  {'explanation': 'The review indicates a somewhat positive experience with a minor negative aspect.',
   'flavor': 'Mango Tango',
   'rating': 1,
   'sentiment': 'NEGATIVE'}]]

## Function calling

[Function Calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) in Gemini lets developers create a description of a function in their code, then pass that description to a language model in a request.

You can submit a Python function for automatic function calling, which will run the function and return the output in natural language generated by Gemini.

You can also submit an [OpenAPI Specification](https://www.openapis.org/) which will respond with the name of a function that matches the description and the arguments to call it with.

### Python Function (Automatic Function Calling)

In [ ]:
def get_current_weather(location: str) -> str:
    """Example method. Returns the current weather.

    Args:
        location: The city and state, e.g. San Francisco, CA
    """
    weather_map: dict[str, str] = {
        "Boston, MA": "snowing",
        "San Francisco, CA": "foggy",
        "Seattle, WA": "raining",
        "Chicago, Il": "windy",
    }

    return weather_map.get(location, "unknown")


response = client.models.generate_content(
    model=MODEL_ID,
    contents="What is the weather like in San Francisco?",
    config=GenerateContentConfig(
        tools=[get_current_weather],
        temperature=0,
    ),
)

display(Markdown(response.text))

The weather in San Francisco is foggy.
